In [1]:
import zarr
import napari
import os
import numpy as np
from skimage.measure import regionprops, regionprops_table
import matplotlib
from matplotlib.cm import ScalarMappable
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
# Load the Zarr data and explore the directory structure
zarr_dir = '../data/data.zarr'
zarr_image_dir = os.path.join(zarr_dir, '0')
zarr_labels_dir = os.path.join(zarr_image_dir, 'labels')

# Image groups
with zarr.open(zarr_dir) as zarr_file:
    print('\033[1mImage groups:\033[0m\n', zarr_file.info)

# Image arrays
with zarr.open(zarr_image_dir) as zarr_file:
    print('\033[1mImage arrays:\033[0m\n', zarr_file.info)

# Labels groups and arrays
with zarr.open(zarr_labels_dir) as zarr_file:
    print('\033[1mLabel groups:\033[0m\n', zarr_file.info)
    label_groups = zarr_file.groups()
    print('\033[1mLabel arrays:\033[0m')
    for group in label_groups:
        print(group[1].info)

# Complete directory structure
!tree $zarr_dir

Image groups:
 Name        : /
Type        : zarr.hierarchy.Group
Read-only   : False
Store type  : zarr.storage.DirectoryStore
No. members : 1
No. arrays  : 0
No. groups  : 1
Groups      : 0

Image arrays:
 Name        : /
Type        : zarr.hierarchy.Group
Read-only   : False
Store type  : zarr.storage.DirectoryStore
No. members : 7
No. arrays  : 6
No. groups  : 1
Arrays      : 0, 1, 2, 3, 4, 5
Groups      : labels

Label groups:
 Name        : /
Type        : zarr.hierarchy.Group
Read-only   : False
Store type  : zarr.storage.DirectoryStore
No. members : 2
No. arrays  : 0
No. groups  : 2
Groups      : 8f296a93-214b-4180-b935-31a0d7406607,
            : e2e7b801-51f5-47e4-b7c8-30ce72cb93d8

Label arrays:
Name        : /8f296a93-214b-4180-b935-31a0d7406607
Type        : zarr.hierarchy.Group
Read-only   : False
Store type  : zarr.storage.DirectoryStore
No. members : 5
No. arrays  : 5
No. groups  : 0
Arrays      : 0, 1, 2, 3, 4

Name        : /e2e7b801-51f5-47e4-b7c8-30ce72cb93d8
Type  

In [3]:
# Retrieve image data and arrange for napari
with zarr.open('../data/data.zarr/0') as data:
    image = [np.moveaxis(data[a][0, 0, :, :, :], 0, -1) for a in data.array_keys()]

# Retrieve label image data and arrange for napari
with zarr.open('../data/data.zarr/0/labels/8f296a93-214b-4180-b935-31a0d7406607') as data:
    label_he = [data[a][0, 0, :, :, :] for a in data.array_keys()]
with zarr.open('../data/data.zarr/0/labels/e2e7b801-51f5-47e4-b7c8-30ce72cb93d8') as data:
    label_fluo = [data[a][0, 0, :, :, :] for a in data.array_keys()]

In [4]:
# VIsualize label images and original image overlay

with napari.gui_qt():
    viewer = napari.view_image(image, multiscale=True, rgb=True)
    viewer.add_labels(label_he, multiscale=True, opacity=0.5, color={v: 'cyan' for v in range(1,10000)})
    viewer.add_labels(label_fluo, multiscale=True, opacity=0.5, color={v: 'yellow' for v in range(1,10000)})

In [5]:
# Measure objects

properties = ['label', 'area', 'mean_intensity', 'eccentricity', 'perimeter']
props_fluo = regionprops_table(label_fluo[0][0, :, :], image[0], properties=properties)
objects_fluo = pd.DataFrame(props_fluo)
objects_fluo['mean_intensity'] = objects_fluo['mean_intensity-0'] * 0.2126 + objects_fluo['mean_intensity-1'] * 0.7152 + objects_fluo['mean_intensity-2'] * 0.0722
num_objects_fluo = len(objects_fluo.index)

objects_fluo

,label,area,mean_intensity-0,mean_intensity-1,mean_intensity-2,eccentricity,perimeter,mean_intensity
0,1,306,48.568627,28.581699,122.509804,0.787270,74.284271,39.612529
1,2,126,44.595238,31.230159,144.833333,0.707666,40.970563,42.273724
2,3,95,87.705263,31.905263,138.536842,0.829216,36.970563,51.467143
3,4,157,40.401274,34.452229,149.961783,0.671685,45.213203,44.056786
4,5,428,74.556075,43.766355,164.560748,0.839016,83.840620,59.033605
...,...,...,...,...,...,...,...,...
4588,5029,208,36.274038,16.375000,110.562500,0.769408,53.213203,27.405873
4589,5030,259,61.745174,36.926641,151.123552,0.529347,58.870058,50.448078
4590,5031,321,54.794393,31.953271,138.118380,0.787603,67.698485,44.474414
4591,5032,192,90.828125,34.729167,147.515625,0.332364,48.627417,54.798987


In [8]:
# Build color maps

color_dict = {p: {} for p in properties}
for p in properties:
    min_p = np.percentile(objects_fluo[p], 10)
    max_p = np.percentile(objects_fluo[p], 90)
    norm = matplotlib.colors.Normalize(vmin=min_p, vmax=max_p)
    mapper = ScalarMappable(norm=norm, cmap=matplotlib.cm.magma)
    color_dict[p] = {
        n: list(
            mapper.to_rgba(
                objects_fluo.loc[objects_fluo['label'] == n, [p]].values[0][0])) for n in objects_fluo['label']}

In [9]:
# Visualize with color map based on area

with napari.gui_qt():
    viewer = napari.view_image(image, multiscale=True, rgb=True)
    viewer.add_labels(label_fluo, multiscale=True, opacity=0.5, color=color_dict['mean_intensity'])

In [10]:
# Cluster cells into types

data = []
for n in range(num_objects_fluo):
    data.append([objects_fluo[p][n] for p in properties if p != 'label'])
n_clusters = 3
km = KMeans(n_clusters=n_clusters)
km.fit(data)
prediction = km.predict(data)

color_dict = {}
norm = matplotlib.colors.Normalize(vmin=0, vmax=n_clusters - 1)
mapper = matplotlib.cm.ScalarMappable(norm=norm, cmap=matplotlib.cm.rainbow)
for n in objects_fluo['label'].values:
    m = list(objects_fluo['label'].values).index(n)
    color_dict[n] = list(mapper.to_rgba(prediction[m]))

In [11]:
# Visualize clusters of TRAINING data

with napari.gui_qt():
    viewer = napari.view_image(image, multiscale=True, rgb=True)
    viewer.add_labels(label_fluo, multiscale=True, opacity=1, color=color_dict)